<a href="https://colab.research.google.com/github/jeswanth1212/loan-status-predictor/blob/main/Mission_Impossible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from collections import Counter

In [ ]:
train_df = pd.read_csv('DMI_train.csv')

train_df.head()

,Index,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,class
0,1,1!.2376@6^07!🅰️46🏧2203^2(84%e*🇦🇱+1🥇8,301.44!9!75&1+🥉0🥇517🥉0🏧56$,27_.0)40724*🥈615478@9@,2🥇4.7*🥇69@58),23.^🥉08!🇦🇱7%9*,21).#4#80&66🥉,20$🇦🇱.47^4#2🥇19,1🥉9🥈.7#🆎1^🇦🇱9🏧647@,360$9,301+,5_🅰️,1#2^6,1(0🅰️8701*87.🥉0#,45🆎3%,1🅰.2!70🅰0_7🅰️272+37$1*5🆎7)2,83🇩🇿5)2,57^26🇩🇿0,LIE
1,2,1*.^23767)112🇦🇱7*60_5^70🥉7e^+18🅰,20🇦🇱5.550🏧5#1&8784🅰️6%🥈5#1🏧22,31).0$483_4(264101(91$1!,20.🥈4(1(🥈4^29,21🇦🇫._🆎1🏧98🅰9*1#,20.&487(3+3,19🅰.7^586^71,19.8)86🅰️14&5,NaN,3🥈01🥈,4,56(,12@775🇦🇱440^.!0,29,1+.5^3(44*94_667🇦🇱3)83^49$,73#5🥇8,56(7🅰️63,TRUTH
2,3,1.&🅰️23@7^6!6🥇0&1%52#71%556+e+1)8🇩🇿,83@.241+🥇1833🏧83#3@806🇩🇿3,1*2).6^🥇86🅰️924757^🇦🇫07$🏧378🆎6,22🆎.)31!9)79🅰,22.9🥇90🥇32$,2@🥇2.3🅰️9039!,2+🅰1.904*91🆎5,2🥇1.&5🆎25@323),3🅰️225)🏧,301,5🇦🇫,158!🅰️,2^🥈8!8🆎2231*.0,59@,2🥉.🥇54*3+396🥇9_2🆎1$🥈6435!🥈5🥈,8466_,5)🥈7!402,TRUTH
3,4,1*🇦🇱.2$376728^7713&🇩🇿30^3!7e🇦🇫+🇦🇫18,33@🇦🇫6.6_5*95^8^🅰️8%1^59078_3*4,4^1^.1%2🥈7!4#56339@157085(🇦🇱,21🇩🇿.+🥈39+76!5$,2_1.735!93^🅰,1🥇9.98🥈7)🥈99$,19(🇦🇱.3$9*7&7(3🅰️9🇩🇿,18.93(🥇3)2_5,7080,3^01🇦🇫,5,2🅰️8+8*,1_0^0617&48.0*,8&🇦🇱5$3🇦🇱,1*.3🥉62🇦🇫7170)4!37*83🅰41,3696,54938🆎,TRUTH
4,5,1#.(2🥈37)6+6$223616🇦🇱580&4(3$e+1!🏧8@🇦🇱,15.80^1🅰️0🥉1🇦🇱4🅰️136604@🥈4🅰️8*3,1+0.🇦🇱11@6&55)🥈53180🆎92!9)42#🏧,22.🇦🇫901🅰️89+,21🆎.(5^1🅰5$🅰️08,19(🅰.44^🅰️96+6,1🇦🇫7!.9🇩🇿086$11,1*🅰7*.^3🆎7🥇71(12,43!94,30@1%,1,74@,3🆎0+🇩🇿00$801.(0),1%42,0_🏧.265%04!🥇4%9411🥉40)🇦🇱26*,2🥉5(0🏧0,548%6#🇩🇿4),TRUTH


In [ ]:
import pandas as pd
import re
# Load datasets with alternative encoding and handle bad lines
train_df = pd.read_csv('DMI_train.csv', encoding='ISO-8859-1')
test_df = pd.read_csv('DMI_test_user.csv', encoding='ISO-8859-1', on_bad_lines='skip')


In [ ]:
import pandas as pd
import re

# Load datasets with alternative encoding and handle bad lines
train_df = pd.read_csv('DMI_train.csv', encoding='ISO-8859-1')
test_df = pd.read_csv('DMI_test_user.csv', encoding='ISO-8859-1', on_bad_lines='skip')

# Separate features (X_train) and target (y_train)
X_train = train_df.drop(columns=['class'])
y_train = train_df['class'].replace({'TRUTH': 0, 'LIE': 1, 'SUS': 2})

# Function to clean text: keep only numbers and dots
def clean_text(text):
    return re.sub(r'[^0-9.]', '', str(text))

# Apply the cleaning process to both train and test data
X_train_cleaned = X_train.applymap(clean_text).apply(pd.to_numeric, errors='coerce')
X_test_cleaned = test_df.applymap(clean_text).apply(pd.to_numeric, errors='coerce')

# Fill missing values with column mean instead of dropping them
X_train_cleaned = X_train_cleaned.apply(lambda col: col.fillna(col.mean()))
X_test_cleaned = X_test_cleaned.apply(lambda col: col.fillna(col.mean()))

# Reset index for clean data
X_train_cleaned = X_train_cleaned.reset_index(drop=True)
y_train_cleaned = y_train.reset_index(drop=True)
X_test_cleaned = X_test_cleaned.reset_index(drop=True)

# Print cleaned features
print("Cleaned X_train:")
print(X_train_cleaned.head())

print("y_train:")
print(y_train_cleaned.head())

print("Cleaned X_test:")
print(X_test_cleaned.head())


<ipython-input-39-ccb090fee9c3>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = train_df['class'].replace({'TRUTH': 0, 'LIE': 1, 'SUS': 2})
<ipython-input-39-ccb090fee9c3>:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X_train_cleaned = X_train.applymap(clean_text).apply(pd.to_numeric, errors='coerce')
<ipython-input-39-ccb090fee9c3>:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X_test_cleaned = test_df.applymap(clean_text).apply(pd.to_numeric, errors='coerce')


Cleaned X_train:
   Index        f1          f2         f3        f4        f5        f6  \
0      1  1.237661  301.449751  27.040725  24.76958  23.08790  21.48066   
1      2  1.237671  205.550519  31.048343  20.41429  21.19891  20.48733   
2      3  1.237660   83.241183  12.686925  22.31979  22.99032  22.39039   
3      4  1.237673  336.659588  41.127456  21.39765  21.73593  19.98799   
4      5  1.237662   15.801014  10.116555  22.90189  21.51508  19.44966   

          f7         f8           f9    f10  f11    f12         f13    f14  \
0  20.474219  19.719647  3609.000000  301.0  5.0  126.0  10870187.0  453.0   
1  19.758671  19.886145  4741.816945  301.0  4.0   56.0  12775440.0   29.0   
2  21.904915  21.525323  3225.000000  301.0  5.0  158.0   2882231.0   59.0   
3  19.397739  18.933250  7080.000000  301.0  5.0  288.0  10061748.0  853.0   
4  17.908611  17.377112  4394.000000  301.0  1.0   74.0   3000801.0  142.0   

        f15     f16      f17  
0  1.270073  8352.0  57260.0  
1

In [ ]:
# Get the number of rows and columns in X_train_cleaned
rows, columns = train_df.shape

print(f'Number of rows: {rows}')
print(f'Number of columns: {columns}')


Number of rows: 702770
Number of columns: 19


In [ ]:
# Get the number of rows and columns in X_train_cleaned
rows, columns = X_test_cleaned.shape

print(f'Number of rows: {rows}')
print(f'Number of columns: {columns}')


Number of rows: 302800
Number of columns: 18


In [ ]:
# Get the number of rows and columns in X_train_cleaned
rows, columns = test_df.shape

print(f'Number of rows: {rows}')
print(f'Number of columns: {columns}')


Number of rows: 302800
Number of columns: 18


In [ ]:
# This will return a tuple where the first value is the number of rows
num_rows = y_train_cleaned.shape[0]
print(f"Number of rows: {num_rows}")


Number of rows: 700000


In [ ]:
# Get the number of rows and columns in X_train_cleaned
rows, columns = X_train.shape

print(f'Number of rows: {rows}')
print(f'Number of columns: {columns}')


Number of rows: 700000
Number of columns: 18


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Encode the target labels for y_train_cleaned if they are not numeric already
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_cleaned)

# Split the data into training (80%) and testing sets (20%)
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train_cleaned, y_train_encoded, test_size=0.2, random_state=42)

# Initialize the XGBoost classifier with hyperparameters
xgb_model = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    learning_rate=0.05,
    n_estimators=400,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0.1,
    reg_lambda=1,
    reg_alpha=0,
    objective='multi:softmax',  # Multi-class classification
    num_class=3  # Based on the number of classes (0, 1, 2)
)

# Train the XGBoost model
xgb_model.fit(X_train_split, y_train_split)

# Make predictions on the test data
y_pred_test = xgb_model.predict(X_test_split)

# Evaluate model performance on test data
accuracy_test = accuracy_score(y_test_split, y_pred_test)
print(f"Test accuracy: {accuracy_test * 100:.2f}%")
print("Test Classification Report:\n", classification_report(y_test_split, y_pred_test))

# Predictions on X_test_cleaned
y_pred_final = xgb_model.predict(X_test_cleaned)

# Reverse label encoding for the final predictions (if needed)
y_pred_final_decoded = label_encoder.inverse_transform(y_pred_final)

# Prepare the submission DataFrame with 'ID' and 'Prediction'
submission_df = X_test_cleaned[['index']].copy()  # Assuming 'index' is the ID column
submission_df['Prediction'] = y_pred_final_decoded  # Add predictions

# Save to CSV
submission_df.to_csv('submission.csv', index=False)

print("Submission file generated: submission.csv")
